In [ ]:
import numpy as np
import xarray as xr
from typing import Union
from pathlib import Path

In [ ]:
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
import dask.array as da

In [ ]:
# cluster = LocalCluster()
# client = Client(cluster)
# client

## Preprocess

Add time bounds with something like
```
for year in {1985..2019}; do
    cdo -O -f nc4 -z zip_2 -settbounds,year -setdate,${year}-1-1 -setreftime,${year}-1-1 GRE_G0240_${year}.nc GRE_G0240_T_${year}.nc; 
done
```

In [ ]:
ds = xr.open_mfdataset("~/base/pism-ragis/data/its_live/GRE_G0240_T_*.nc", chunks={"x": -1, "y": -1, "time": -1})
vars_to_process = ["v", "vx", "vy", "v_err", "vx_err", "vy_err", "ice"]
ds = ds[vars_to_process]
ice_ds = ds.where(ds["ice"])

In [ ]:
jak_ds = ice_ds.sel({"x": slice(-180_000, -80_000), "y": slice(-2_200_000, -2_300_000)})
jak_speed = jak_ds["v"]

In [ ]:
nt = ds.time.size
dt = xr.DataArray(
    da.arange(nt, chunks=-1),
    dims=("time"),
)

In [ ]:
def idw_weights(distance: xr.DataArray, power: float = 1.0):
    return 1.0 / (distance + 1e-12) ** power    

In [ ]:
def save(ds: xr.Dataset, output_filename: Union[str, Path] = "GRE_G0240_1985_2018_IDW_EXP_1.nc" ,comp = dict(zlib=True, complevel=2)
):
    encoding = {var: comp for var in ds.data_vars}
    ds.to_netcdf(output_filename, encoding=encoding)


In [ ]:
nt = ds.time.size
power = 1

distance = np.isfinite(jak_speed) * dt.broadcast_like(jak_speed)
weights = idw_weights(distance, power=power)

In [ ]:
with ProgressBar():
    jak_wm = jak_speed_weighted_mean = jak_ds.weighted(weights).mean(dim="time")
    save(jak_wm, "jak_idw.nc")

In [ ]:
with ProgressBar():
    jak_speed_mean = jak_speed.mean(dim="time").compute()

In [ ]:
distance = np.isfinite(speed) * dt.broadcast_like(speed)
weights = idw_weights(distance, power=power)
with ProgressBar():
    speed_weighted_mean = ice_ds.weighted(weights).mean(dim="time")
    save(speed_weighted_mean)

In [ ]:
speed_weighted_mean.plot(vmax=1500)

In [ ]:
ice_ds = ds.where(ds["ice"])
speed = ice_ds["v"]
distance = np.isfinite(speed) * dt.broadcast_like(ice_ds)
weights = idw_weights(distance, power=power)
with ProgressBar():
    weighted_mean = ice_ds.weighted(weights).mean(dim="time").compute()

In [ ]:
weigts

In [ ]:
ice_ds